In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import make_moons
from tensorflow.keras import regularizers

In [2]:
data = make_moons(3000, noise=0.05)[0].astype("float32")
norm = layers.experimental.preprocessing.Normalization()
norm.adapt(data)
normalized_data = norm(data)